# Karar Ağacı Algoritması (ID3)

### Özel Kütüphane Kullanmadan KNN Algoritmasını Gerçekleştirme

İlk önce Numpy ve Pandas ve diğer gerekli kütüphaneleri çalışmamıza ekleyelim.

In [66]:
import numpy as np
import pandas as pd
from pprint import pprint
from collections import Counter
import math

Ardından play_tennis veri setimizi Pandas kütüphanesi yardımıyla çalışmamıza ekleyelim ve oluşturduğumuz dataframe'i karıştıralım.

In [67]:
df = pd.read_csv("play_tennis.csv")
df = df.sample(frac=1)  # Dataframe'i karıştıralım.
df

,Outlook,Temp,Humidity,Wind,PlayTennis
2,Overcast,Hot,High,Weak,Yes
11,Overcast,Mild,High,Strong,Yes
7,Sunny,Mild,High,Weak,No
3,Rain,Mild,High,Weak,Yes
10,Sunny,Mild,Normal,Strong,Yes
13,Rain,Mild,High,Strong,No
12,Overcast,Hot,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes
6,Overcast,Cool,Normal,Strong,Yes
8,Sunny,Cool,Normal,Weak,Yes


<img src="https://www.ibm.com/content/dam/connectedassets-adobe-cms/worldwide-content/sys/cf/ul/g/b5/8c/Entropy-Formula.component.xl.ts=1640286171390.png/content/adobe-cms/us/en/topics/decision-trees/jcr:content/root/table_of_contents/body/complex_narrative_44780520/items/content_group_86925341/image" width="600" height="400">

Yukarıda verilen Entropi formulününü hesaplayan bir fonksiyon yazalım:

In [68]:
def entropy(probs):  
    return sum( [-prob*math.log(prob, 2) for prob in probs])

def entropy_of_list(ls,value):  
    from collections import Counter
    
    total_instances = len(ls)

    cnt = Counter(x for x in ls)

    probs = [x / total_instances for x in cnt.values()]  
    
    return entropy(probs) 

<img src="https://cdn-images-1.medium.com/v2/resize:fit:800/1*Lr8007IV19DRUZZCpMDWAg.png">

Yukarıda verilen Bilgi Kazanımı formulününü hesaplayan bir fonksiyon yazalım:

In [69]:
def information_gain(df, split_attribute, target_attribute,battr):
    
    df_split = df.groupby(split_attribute) 
    glist=[]
    for gname,group in df_split:
        glist.append(gname) 
    
    glist.reverse()
    nobs = len(df.index) * 1.0   
    df_agg1=df_split.agg({target_attribute:lambda x:entropy_of_list(x, glist.pop())})
    df_agg2=df_split.agg({target_attribute :lambda x:len(x)/nobs})
    
    df_agg1.columns=['Entropy']
    df_agg2.columns=['Proportion']
    
    new_entropy = sum( df_agg1['Entropy'] * df_agg2['Proportion'])
    if battr !='S':
        old_entropy = entropy_of_list(df[target_attribute],'S-'+df.iloc[0][df.columns.get_loc(battr)])
    else:
        old_entropy = entropy_of_list(df[target_attribute],battr)
    return old_entropy - new_entropy

In [70]:
def id3(df, target_attribute, attribute_names, default_class=None,default_attr='S'):
    
    cnt = Counter(x for x in df[target_attribute])
    
    if len(cnt) == 1:
        return next(iter(cnt)) 

    elif df.empty or (not attribute_names):
        return default_class 
    
    else:
        default_class = max(cnt.keys())
        gainz=[]
        for attr in attribute_names:
            ig= information_gain(df, attr, target_attribute,default_attr)
            gainz.append(ig)
        
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max] 

        tree = {best_attr:{}}
        remaining_attribute_names =[i for i in attribute_names if i != best_attr]
        
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute, remaining_attribute_names,default_class,best_attr)
            tree[best_attr][attr_val] = subtree
        return tree

In [71]:
def entropy_dataset(a_list):  
    from collections import Counter

    cnt = Counter(x for x in a_list)   
    num_instances = len(a_list)*1.0 
    
    probs = [x / num_instances for x in cnt.values()]  
    
    return entropy(probs) 
    

total_entropy = entropy_dataset(df['PlayTennis'])

from pprint import pprint
tree = id3(df,t,attribute_names)
print("\nOluşturulan Karar Ağacının Son Hali Aşağıdaki Gibidir:\n")
pprint(tree)

attribute = next(iter(tree))


Oluşturulan Karar Ağacının Son Hali Aşağıdaki Gibidir:

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


Oluşturduğumuz ağacı test edelim. Bu test için "play_tennis_test.csv" dosyasını kullanalım. 

İlk örneğimize baktığımızda "Outlook = Sunny" ve "Humidity = Normal" olduğu için ağacımıza göre "PlayTennis = Yes" olması gerekiyor. 

İkinci örneğimiz baktığımızda "Outlook = Overcast" olduğu için ağacımıza göre "PlayTennis = Yes" olması gerekiyor.

Üçüncü örneğimiz baktığımızda "Outlook = Rain" ve "Wind = Strong" olduğu için ağacımıza göre "PlayTennis = No" olması gerekiyor.

In [72]:
def classify(instance, tree,default=None): 
    attribute = next(iter(tree))  
    if instance[attribute] in tree[attribute].keys(): 
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict): 
            return classify(instance, result)
        else:
            return result
    else:
        return default
    
df_new=pd.read_csv('play_tennis_test.csv')
df_new['Predicted'] = df_new.apply(classify, axis=1, args=(tree,'?')) 
print(df_new)

    Outlook Temp Humidity    Wind PlayTennis Predicted
0     Sunny  Hot   Normal    Weak          ?       Yes
1  Overcast  Hot     High  Strong          ?       Yes
2      Rain  Hot     High  Strong          ?        No


### Sckit-Learn Kütüphanesi Kullanarak KNN Algoritmasını Gerçekleştirme

In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [62]:
df = pd.read_csv("Iris.csv")
df.drop("Id",inplace=True,axis=1) # Model için kullanmayacağımız sütunları dataframe'den çıkaralım.
df = df.sample(frac=1) # Dataframe'i karıştıralım.
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
87,6.3,2.3,4.4,1.3,Iris-versicolor
19,5.1,3.8,1.5,0.3,Iris-setosa
148,6.2,3.4,5.4,2.3,Iris-virginica
120,6.9,3.2,5.7,2.3,Iris-virginica
108,6.7,2.5,5.8,1.8,Iris-virginica
...,...,...,...,...,...
37,4.9,3.1,1.5,0.1,Iris-setosa
89,5.5,2.5,4.0,1.3,Iris-versicolor
34,4.9,3.1,1.5,0.1,Iris-setosa
99,5.7,2.8,4.1,1.3,Iris-versicolor


In [63]:
X = np.array(df.iloc[:,0:-1])
y = np.array(df.iloc[:,-1:]).reshape((-1,))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [64]:
dtree = DecisionTreeClassifier(criterion='entropy',max_depth=3)
dtree.fit(X_train,y_train)
y_pred = dtree.predict(X_test)
accuracy_score(y_test, y_pred)

0.9111111111111111